In [595]:
import pandas as pd 

# URL del archivo Excel en formato raw
url = 'https://github.com/unir-henry/henry/raw/main/Apidae_iNaturalist_LIMPIA.xlsx'

# Leer el archivo Excel
df = pd.read_excel(url, engine='openpyxl', sheet_name=0)

# Crear una nueva columna 'Ubicacion' que unifique las columnas deseadas
df['Ubicacion'] = df[['Ubicación', 1, 2, 3, 4, 5, 6, 7, 8]].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

# Limpiar la nueva columna 'Ubicacion' para asegurarnos de que no haya caracteres no deseados
df['Ubicacion'] = df['Ubicacion'].str.encode('utf-8', 'ignore').str.decode('utf-8')

# Eliminar la palabra "Ecuador" de la columna 'Ubicacion'
df['Ubicacion'] = df['Ubicacion'].str.replace('Ecuador', '', regex=False)

# Quitar espacios en blanco al inicio y al final de la columna 'Ubicacion'
df['Ubicacion'] = df['Ubicacion'].str.strip()

# Seleccionar las columnas deseadas: 'Ubicacion', 'Latitud', 'Longitud', 'Familia', 'Genero-ESPECIE'
nuevo_df = df[['Ubicacion', 'Latitud', 'Longitud', 'Familia', 'Genero-ESPECIE']]

# Mostrar el nuevo DataFrame
print(nuevo_df)



                                    Ubicacion   Latitud   Longitud Familia  \
0                               Sucre  Manabi -0.105481 -80.150883  Apidae   
1                               Sucre  Manabi -0.105717 -80.150756  Apidae   
2       Jama  Canton of Jama  Manabi Province -0.183330 -80.266670  Apidae   
3                                isla isabela -0.829237 -91.135302  Apidae   
4                  Santa Cruz  GalÃ¡pagos  EC -0.741832 -90.308680  Apidae   
...                                       ...       ...        ...     ...   
6660  MATILDE ALVAREZ Y Oe4 (1)  170146 Quito -0.285283 -78.552277  Apidae   
6661                             zona1  Quito -0.167853 -78.482331  Apidae   
6662                                Latacunga -0.753611 -78.664974  Apidae   
6663                                Tanicuchi -0.753790 -78.664810  Apidae   
6664                        9V29+F48  Palanda -4.648836 -79.132267  Apidae   

              Genero-ESPECIE  
0             Apis mellifera  
1

In [597]:
import pandas as pd

# URL del archivo Excel en formato raw
url_ubicaciones = 'https://github.com/unir-henry/henry/raw/main/Ubicaciones.xlsx'

# Leer el archivo Excel
df_ubicaciones = pd.read_excel(url_ubicaciones, engine='openpyxl')

# Quitar espacios en blanco de todas las columnas
df_ubicaciones = df_ubicaciones.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Convertir todos los datos a minúsculas y poner la primera letra de cada palabra en mayúscula
df_ubicaciones = df_ubicaciones.applymap(lambda x: x.title() if isinstance(x, str) else x)

# Mostrar las primeras filas del DataFrame
print(df_ubicaciones.head())



  Provincia  Canton    Parroquia
0     Azuay  Cuenca   Bellavista
1     Azuay  Cuenca  Cañaribamba
2     Azuay  Cuenca     El Batán
3     Azuay  Cuenca  El Sagrario
4     Azuay  Cuenca    El Vecino


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_15512\2989447434.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_ubicaciones = df_ubicaciones.applymap(lambda x: x.strip() if isinstance(x, str) else x)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_15512\2989447434.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_ubicaciones = df_ubicaciones.applymap(lambda x: x.title() if isinstance(x, str) else x)


In [599]:
import pandas as pd

# Supongamos que df_ubicaciones ya está definido
# Eliminar filas donde la Parroquia sea "Ecuador"
df_ubicaciones = df_ubicaciones.drop(df_ubicaciones[df_ubicaciones['Parroquia'] == "Ecuador"].index)

# Mostrar el DataFrame resultante
print(df_ubicaciones)


                 Provincia           Canton                Parroquia
0                    Azuay           Cuenca               Bellavista
1                    Azuay           Cuenca              Cañaribamba
2                    Azuay           Cuenca                 El Batán
3                    Azuay           Cuenca              El Sagrario
4                    Azuay           Cuenca                El Vecino
...                    ...              ...                      ...
1393           Santa Elena          Salinas                Anconcito
1394           Santa Elena          Salinas  José Luis Tamayo (Muey)
1395  Zonas No Delimitadas  Las Golondrinas          Las Golondrinas
1396  Zonas No Delimitadas   Manga Del Cura           Manga Del Cura
1397  Zonas No Delimitadas      El Piedrero              El Piedrero

[1397 rows x 3 columns]


In [640]:
import pandas as pd
from fuzzywuzzy import process, fuzz

# Asegúrate de que df_ubicaciones esté definido y contenga las columnas necesarias
# df_ubicaciones = pd.read_excel('tu_archivo.xlsx')  # Descomenta y ajusta si es necesario

# Crear lista de Provincias y un diccionario de Provincias a Provincias
provincias = df_ubicaciones['Provincia'].tolist()
provincias_normalizadas = [provincia.strip().lower() for provincia in provincias]  # Normaliza provincias

# Función para buscar coincidencias en la columna Ubicacion
def buscar_provincia_en_ubicacion(row):
    if not row['Ubicacion']:  # Comprobar si la ubicación está vacía
        return None

    ubicacion_normalizada = row['Ubicacion'].strip().lower()  # Normaliza la ubicación

    # Comparar con Provincias
    match = process.extractOne(ubicacion_normalizada, provincias_normalizadas, scorer=fuzz.token_set_ratio)
    if match and match[1] >= 80:  # Ajustar el umbral si es necesario
        return match[0]  # Retorna la provincia si hay coincidencia

    return None  # Retornar None si no hay coincidencias

# Aplicar la función y crear la nueva columna
nuevo_df['provinciaOK'] = nuevo_df.apply(buscar_provincia_en_ubicacion, axis=1)

# Guardar el nuevo DataFrame en memoria
dataset_provincia = nuevo_df[['Ubicacion', 'Latitud', 'Longitud', 'Familia', 'Genero-ESPECIE', 'provinciaOK']]

# Mostrar el contenido del DataFrame
print(dataset_provincia)


                                    Ubicacion   Latitud   Longitud Familia  \
0                               Sucre  Manabi -0.105481 -80.150883  Apidae   
1                               Sucre  Manabi -0.105717 -80.150756  Apidae   
2       Jama  Canton of Jama  Manabi Province -0.183330 -80.266670  Apidae   
3                                isla isabela -0.829237 -91.135302  Apidae   
4                  Santa Cruz  GalÃ¡pagos  EC -0.741832 -90.308680  Apidae   
...                                       ...       ...        ...     ...   
6660  MATILDE ALVAREZ Y Oe4 (1)  170146 Quito -0.285283 -78.552277  Apidae   
6661                             zona1  Quito -0.167853 -78.482331  Apidae   
6662                                Latacunga -0.753611 -78.664974  Apidae   
6663                                Tanicuchi -0.753790 -78.664810  Apidae   
6664                        9V29+F48  Palanda -4.648836 -79.132267  Apidae   

              Genero-ESPECIE provinciaOK  
0             Apis m

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_15512\4092472608.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nuevo_df['provinciaOK'] = nuevo_df.apply(buscar_provincia_en_ubicacion, axis=1)


In [643]:
# Guardar el nuevo dataframe en un archivo Excel
dataset_provincia.to_excel('con_provinciaOK.xlsx', index=False)


In [647]:
import pandas as pd
from fuzzywuzzy import process, fuzz

# Asegúrate de que df_ubicaciones esté definido y contenga las columnas necesarias
# df_ubicaciones = pd.read_excel('tu_archivo.xlsx')  # Descomenta y ajusta si es necesario

# Crear lista de Cantones y un diccionario de Cantones a Provincias
cantones = df_ubicaciones['Canton'].tolist()
canton_provincia = df_ubicaciones.set_index('Canton')['Provincia'].to_dict()

# Normaliza los cantones para que sean todos minúsculas
cantones_normalizados = [canton.strip().lower() for canton in cantones]
canton_provincia_normalizado = {canton.strip().lower(): provincia for canton, provincia in canton_provincia.items()}

# Función para buscar la provincia correspondiente al canton en la columna Ubicacion
def buscar_provincia_en_ubicacion(row):
    # Solo proceder si la columna 'provinciaOK' está vacía
    if pd.isna(row['provinciaOK']):
        ubicacion_normalizada = row['Ubicacion'].strip().lower()  # Normaliza la ubicación
        # Busca el canton en la ubicación
        match = process.extractOne(ubicacion_normalizada, cantones_normalizados, scorer=fuzz.token_set_ratio)

        if match and match[1] >= 80:  # Ajusta el umbral si es necesario
            provincia = canton_provincia_normalizado.get(match[0])  # Obtiene la provincia
            return provincia

    return row['provinciaOK']  # Retornar el valor actual de provinciaOK si ya está lleno

# Aplicar la función para buscar provincias en la columna 'Ubicacion'
nuevo_df['provinciaOK'] = nuevo_df.apply(buscar_provincia_en_ubicacion, axis=1)

# Guardar el nuevo DataFrame en memoria
dataset_procanton = nuevo_df[['Ubicacion', 'Latitud', 'Longitud', 'Familia', 'Genero-ESPECIE', 'provinciaOK']]

# Mostrar el contenido del DataFrame
print(dataset_procanton)


                                    Ubicacion   Latitud   Longitud Familia  \
0                               Sucre  Manabi -0.105481 -80.150883  Apidae   
1                               Sucre  Manabi -0.105717 -80.150756  Apidae   
2       Jama  Canton of Jama  Manabi Province -0.183330 -80.266670  Apidae   
3                                isla isabela -0.829237 -91.135302  Apidae   
4                  Santa Cruz  GalÃ¡pagos  EC -0.741832 -90.308680  Apidae   
...                                       ...       ...        ...     ...   
6660  MATILDE ALVAREZ Y Oe4 (1)  170146 Quito -0.285283 -78.552277  Apidae   
6661                             zona1  Quito -0.167853 -78.482331  Apidae   
6662                                Latacunga -0.753611 -78.664974  Apidae   
6663                                Tanicuchi -0.753790 -78.664810  Apidae   
6664                        9V29+F48  Palanda -4.648836 -79.132267  Apidae   

              Genero-ESPECIE       provinciaOK  
0             

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_15512\2138829396.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nuevo_df['provinciaOK'] = nuevo_df.apply(buscar_provincia_en_ubicacion, axis=1)


In [649]:
# Guardar el nuevo dataframe en un archivo Excel
dataset_procanton.to_excel('con_ProvCanton.xlsx', index=False)

In [653]:
import pandas as pd
from fuzzywuzzy import process, fuzz

# Asegúrate de que df_ubicaciones esté definido y tenga las columnas necesarias
# df_ubicaciones = pd.read_excel('tu_archivo.xlsx')  # Carga el DataFrame si es necesario

# Crear lista de Parroquias y un diccionario de Parroquias a Provincias
parroquias = df_ubicaciones['Parroquia'].tolist()
parroquia_provincia = df_ubicaciones.set_index('Parroquia')['Provincia'].to_dict()

# Normaliza las parroquias
parroquias = [p.strip().lower() for p in parroquias]
parroquia_provincia = {p.strip().lower(): provincia for p, provincia in parroquia_provincia.items()}

# Función para buscar coincidencias en Parroquias
def buscar_parroquia(ubicacion):
    if not ubicacion:  # Comprobar si la ubicación está vacía
        return None

    # Normaliza la ubicación
    ubicacion_normalizada = ubicacion.strip().lower()
    match = process.extractOne(ubicacion_normalizada, parroquias, scorer=fuzz.token_set_ratio)
    
    if match and match[1] >= 80:  # Ajustar el umbral si es necesario
        provincia = parroquia_provincia.get(match[0])  # Busca la provincia normalizada
        return provincia

    return None  # Retornar None si no hay coincidencias

# Función para buscar provincias a partir de la ubicación
def buscar_provincia(row):
    # Si provinciaOK ya tiene un valor, retornar ese valor
    if pd.notna(row.get('provinciaOK')) and row['provinciaOK'] != '':
        return row['provinciaOK']
    
    # Buscar por parroquia
    provincia = buscar_parroquia(row['Ubicacion'])
    return provincia  # Retorna la provincia encontrada

# Asegúrate de que nuevo_df esté definido y contenga la columna 'Ubicacion'
# nuevo_df = pd.DataFrame(...)  # Carga el DataFrame si es necesario

# Aplicar la función para buscar Parroquias y crear la nueva columna provinciaOK
nuevo_df['provinciaOK'] = nuevo_df.apply(buscar_provincia, axis=1)

# Guardar el nuevo DataFrame en memoria con el nombre ProvCanParroquia
ProvCanParroquia = nuevo_df[['Ubicacion', 'Latitud', 'Longitud', 'Familia', 'Genero-ESPECIE', 'provinciaOK']]

# Mostrar el contenido del DataFrame
print(ProvCanParroquia)


                                    Ubicacion   Latitud   Longitud Familia  \
0                               Sucre  Manabi -0.105481 -80.150883  Apidae   
1                               Sucre  Manabi -0.105717 -80.150756  Apidae   
2       Jama  Canton of Jama  Manabi Province -0.183330 -80.266670  Apidae   
3                                isla isabela -0.829237 -91.135302  Apidae   
4                  Santa Cruz  GalÃ¡pagos  EC -0.741832 -90.308680  Apidae   
...                                       ...       ...        ...     ...   
6660  MATILDE ALVAREZ Y Oe4 (1)  170146 Quito -0.285283 -78.552277  Apidae   
6661                             zona1  Quito -0.167853 -78.482331  Apidae   
6662                                Latacunga -0.753611 -78.664974  Apidae   
6663                                Tanicuchi -0.753790 -78.664810  Apidae   
6664                        9V29+F48  Palanda -4.648836 -79.132267  Apidae   

              Genero-ESPECIE       provinciaOK  
0             

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_15512\530121484.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nuevo_df['provinciaOK'] = nuevo_df.apply(buscar_provincia, axis=1)


In [655]:
# Guardar el nuevo dataframe en un archivo Excel
ProvCanParroquia.to_excel('con_ProvCantonParroquia.xlsx', index=False)

In [697]:
import pandas as pd
from fuzzywuzzy import process, fuzz

# Asegúrate de que df_ubicaciones esté definido y contenga las columnas necesarias
# df_ubicaciones = pd.read_excel('tu_archivo.xlsx')  # Carga el DataFrame si es necesario

# Crear lista de Cantones y Provincias, y normalizar
cantones_provincia = df_ubicaciones[['Canton', 'Provincia']].drop_duplicates()
cantones_provincia['Canton'] = cantones_provincia['Canton'].str.strip().str.lower()  # Normaliza cantones
cantones_provincia['Provincia'] = cantones_provincia['Provincia'].str.strip().str.lower()  # Normaliza provincias

# Función para buscar coincidencias en la columna Ubicacion
def buscar_canton_en_ubicacion(row):
    if not row['Ubicacion']:  # Comprobar si la ubicación está vacía
        return None

    ubicacion_normalizada = row['Ubicacion'].strip().lower()  # Normaliza la ubicación

    # Filtrar cantones por la provincia en provinciaOK
    provincia_actual = row['provinciaOK'].strip().lower()  # Normaliza la provincia
    cantones_filtrados = cantones_provincia[cantones_provincia['Provincia'] == provincia_actual]

    # Comparar con Cantones filtrados
    match = process.extractOne(ubicacion_normalizada, cantones_filtrados['Canton'], scorer=fuzz.token_set_ratio)
    if match and match[1] >= 80:  # Ajustar el umbral si es necesario
        return match[0]  # Retorna el cantón si hay coincidencia

    return None  # Retornar None si no hay coincidencias

# Aplicar la función y crear la nueva columna CantonOK solo si la provinciaOK tiene un valor
nuevo_df['CantonOK'] = nuevo_df.apply(lambda row: buscar_canton_en_ubicacion(row) if pd.notna(row['provinciaOK']) else None, axis=1)

# Guardar el nuevo DataFrame en memoria
dataset_canton = nuevo_df[['Ubicacion', 'Latitud', 'Longitud', 'Familia', 'Genero-ESPECIE', 'provinciaOK', 'CantonOK']]

# Mostrar el contenido del DataFrame
print(dataset_canton)


                                    Ubicacion   Latitud   Longitud Familia  \
0                               Sucre  Manabi -0.105481 -80.150883  Apidae   
1                               Sucre  Manabi -0.105717 -80.150756  Apidae   
2       Jama  Canton of Jama  Manabi Province -0.183330 -80.266670  Apidae   
3                                isla isabela -0.829237 -91.135302  Apidae   
4                  Santa Cruz  GalÃ¡pagos  EC -0.741832 -90.308680  Apidae   
...                                       ...       ...        ...     ...   
6660  MATILDE ALVAREZ Y Oe4 (1)  170146 Quito -0.285283 -78.552277  Apidae   
6661                             zona1  Quito -0.167853 -78.482331  Apidae   
6662                                Latacunga -0.753611 -78.664974  Apidae   
6663                                Tanicuchi -0.753790 -78.664810  Apidae   
6664                        9V29+F48  Palanda -4.648836 -79.132267  Apidae   

              Genero-ESPECIE       provinciaOK    CantonOK  
0 

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_15512\3704046986.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nuevo_df['CantonOK'] = nuevo_df.apply(lambda row: buscar_canton_en_ubicacion(row) if pd.notna(row['provinciaOK']) else None, axis=1)


In [699]:
# Guardar el nuevo dataframe en un archivo Excel
dataset_canton.to_excel('con_CampoCanton.xlsx', index=False)

In [703]:
import pandas as pd
from fuzzywuzzy import process, fuzz

# Asegúrate de que df_ubicaciones esté definido y contenga las columnas necesarias
# df_ubicaciones = pd.read_excel('tu_archivo.xlsx')  # Carga el DataFrame si es necesario

# Crear lista de Cantones y Provincias, y normalizar
cantones_provincia = df_ubicaciones[['Canton', 'Provincia']].drop_duplicates()
cantones_provincia['Canton'] = cantones_provincia['Canton'].str.strip().str.lower()  # Normaliza cantones
cantones_provincia['Provincia'] = cantones_provincia['Provincia'].str.strip().str.lower()  # Normaliza provincias

# Crear lista de Parroquias y normalizar
parroquias = df_ubicaciones[['Parroquia', 'Canton']].drop_duplicates()
parroquias['Parroquia'] = parroquias['Parroquia'].str.strip().str.lower()  # Normaliza parroquias

# Función para buscar canton en base a la ubicación
def buscar_canton_en_ubicacion(row):
    if not row['Ubicacion']:  # Comprobar si la ubicación está vacía
        return None

    ubicacion_normalizada = row['Ubicacion'].strip().lower()  # Normaliza la ubicación

    # Filtrar cantones por la provincia en provinciaOK
    provincia_actual = row['provinciaOK']
    if pd.isna(provincia_actual) or not isinstance(provincia_actual, str):
        return None  # Retornar None si provinciaOK no es válido

    provincia_actual = provincia_actual.strip().lower()  # Normaliza la provincia
    cantones_filtrados = cantones_provincia[cantones_provincia['Provincia'] == provincia_actual]

    # Comparar con Cantones filtrados
    match = process.extractOne(ubicacion_normalizada, cantones_filtrados['Canton'], scorer=fuzz.token_set_ratio)
    if match and match[1] >= 80:  # Ajustar el umbral si es necesario
        return match[0]  # Retorna el cantón si hay coincidencia

    return None  # Retornar None si no hay coincidencias

# Función para buscar canton basado en parroquia
def buscar_canton_por_parroquia(row):
    if pd.notna(row['CantonOK']) and row['CantonOK'] != '':  # Si ya hay un cantón asignado, retorna
        return row['CantonOK']

    if not row['Ubicacion']:  # Comprobar si la ubicación está vacía
        return None

    # Filtrar cantones por la provincia en provinciaOK
    provincia_actual = row['provinciaOK']
    if pd.isna(provincia_actual) or not isinstance(provincia_actual, str):
        return None  # Retornar None si provinciaOK no es válido

    provincia_actual = provincia_actual.strip().lower()  # Normaliza la provincia
    cantones_filtrados = cantones_provincia[cantones_provincia['Provincia'] == provincia_actual]

    # Normaliza la ubicación para buscar parroquias
    ubicacion_normalizada = row['Ubicacion'].strip().lower()

    # Comparar con Parroquias
    match_parroquia = process.extractOne(ubicacion_normalizada, parroquias['Parroquia'], scorer=fuzz.token_set_ratio)
    if match_parroquia and match_parroquia[1] >= 80:  # Ajustar el umbral si es necesario
        canton_asociado = parroquias.loc[parroquias['Parroquia'] == match_parroquia[0], 'Canton'].values
        if len(canton_asociado) > 0:
            return canton_asociado[0]  # Retorna el cantón asociado a la parroquia

    return None  # Retornar None si no hay coincidencias

# Aplicar la función para buscar canton en base a la ubicación
nuevo_df['CantonOK'] = nuevo_df.apply(lambda row: buscar_canton_en_ubicacion(row) if pd.notna(row['provinciaOK']) else None, axis=1)

# Aplicar la función para buscar canton en base a la parroquia solo si CantonOK sigue vacío
nuevo_df['CantonOK'] = nuevo_df.apply(lambda row: buscar_canton_por_parroquia(row), axis=1)

# Guardar el nuevo DataFrame en memoria
dataset_canton = nuevo_df[['Ubicacion', 'Latitud', 'Longitud', 'Familia', 'Genero-ESPECIE', 'provinciaOK', 'CantonOK']]

# Mostrar el contenido del DataFrame
print(dataset_canton)


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_15512\1972905796.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nuevo_df['CantonOK'] = nuevo_df.apply(lambda row: buscar_canton_en_ubicacion(row) if pd.notna(row['provinciaOK']) else None, axis=1)


                                    Ubicacion   Latitud   Longitud Familia  \
0                               Sucre  Manabi -0.105481 -80.150883  Apidae   
1                               Sucre  Manabi -0.105717 -80.150756  Apidae   
2       Jama  Canton of Jama  Manabi Province -0.183330 -80.266670  Apidae   
3                                isla isabela -0.829237 -91.135302  Apidae   
4                  Santa Cruz  GalÃ¡pagos  EC -0.741832 -90.308680  Apidae   
...                                       ...       ...        ...     ...   
6660  MATILDE ALVAREZ Y Oe4 (1)  170146 Quito -0.285283 -78.552277  Apidae   
6661                             zona1  Quito -0.167853 -78.482331  Apidae   
6662                                Latacunga -0.753611 -78.664974  Apidae   
6663                                Tanicuchi -0.753790 -78.664810  Apidae   
6664                        9V29+F48  Palanda -4.648836 -79.132267  Apidae   

              Genero-ESPECIE       provinciaOK    CantonOK  
0 

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_15512\1972905796.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nuevo_df['CantonOK'] = nuevo_df.apply(lambda row: buscar_canton_por_parroquia(row), axis=1)


In [708]:
# Guardar el nuevo dataframe en un archivo Excel
dataset_canton.to_excel('con_CampoCantonParroquia.xlsx', index=False)